In [35]:
import pandas as pd, numpy as np, re, string
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer as tf
train = pd.read_csv("C:\\SpringBoard\\CapstoneOne\\RedditTrain.csv", encoding='utf-8')
test = pd.read_csv("C:\\SpringBoard\\CapstoneOne\\RedditTest.csv", encoding='utf-8')

In [36]:
train.head()

,Unnamed: 0,comment,positive,negative,compund,positive_comment,negative_comment,neutral_comment
0,0,"""You must post a clear and direct question in ...",0.090,0.052,0.1531,1,0,0
1,1,I have learned English.,0.000,0.000,0.0000,0,0,1
2,2,How to bunny hop,0.000,0.000,0.0000,0,0,1
3,3,"""You laugh, but have you ever realized you wal...",0.099,0.000,0.3182,1,0,0
4,4,After getting shot I can heal myself by drinki...,0.189,0.000,0.2732,1,0,0


In [37]:
test.head()

,Unnamed: 0,comment,positive,negative,compund,positive_comment,negative_comment,neutral_comment
0,5298,Yep. Wood=less deadly confirmed,0.423,0.000,0.2960,1,0,0
1,5299,They are discriminating against them based on ...,0.038,0.160,-0.7713,0,1,0
2,5300,Did you just sit here with a straight face and...,0.083,0.131,-0.2748,0,1,0
3,5301,It is less damaging unless you’re talking abou...,0.000,0.188,-0.4601,0,1,0
4,5302,"""I'm not saying I want another mass shooting, ...",0.374,0.029,0.9359,1,0,0


In [39]:
train['comment'][5124]

'They need a complete cultural overhaul.'

In [40]:
stats = train.comment.str.len()
stats.mean(), stats.std(), stats.max()

(130.1136851136851, 124.85501546472132, 1493.0)

In [45]:
#Get rid of empty comments
train['comment'].fillna("unknown",inplace=True)
test['comment'].fillna("unknown", inplace=True)
#label columns
label_cols = ['positive_comment','negative_comment','neutral_comment']

In [46]:
#Build Model creating bag of words representation
re_token = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_token.sub(r'\1',s).split()
n = train.shape[0]
tf
vect = tf(ngram_range=(1,2), tokenizer=tokenize, min_df=3,
                      max_df=.9, strip_accents='unicode',use_idf=1,
                      smooth_idf=1, sublinear_tf=1)
train_term_doc = vect.fit_transform(train['comment'])
test_term_doc = vect.fit_transform(test['comment'])

In [47]:
train_term_doc

<2513x3996 sparse matrix of type '<class 'numpy.float64'>'
	with 45176 stored elements in Compressed Sparse Row format>

In [48]:
test_term_doc

<2513x3996 sparse matrix of type '<class 'numpy.float64'>'
	with 45176 stored elements in Compressed Sparse Row format>

In [56]:
#basic Naive Bayes feature equation
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1)/((y==y_i).sum()+1)
#fit model for one category comment at a time
def model(y):
    y = y.values
    r = np.log(pr(1,y)/pr(0,y))
    m = LogisticRegression(C=4, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [57]:
x = train_term_doc
test_x = test_term_doc

In [58]:
#Don't run till we have test set
preds = np.zeros((len(test_x), len(label_cols)))
for i, j in enumerate(label_cols):
    print('fit',j)
    m,r = model(train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]